In [1]:
using CFET_OrderConditions
using Giac

In [2]:
function legendre(n::Int, x)
    (-1)^(n)*sum([binomial(n,k)*binomial(n+k,k)*(-x)^k for k=0:n])
end

legendre (generic function with 1 method)

In [3]:
function gauss_quadrature_coeffs(n::Int)
    @giac x
    p=simplify(legendre(n,x))   
    t=to_julia(solve(p,x))
    w=[simplify(integrate(prod([(x-t[j])/(t[i]-t[j]) for j=1:n if j!=i]),x,0,1)) for i=1:n]
    t,w
end

gauss_quadrature_coeffs (generic function with 1 method)

In [4]:
function coeffs_a_c(p::Int, J::Int, f, M::Int=div(p,2)+1)
    x,w = gauss_quadrature_coeffs(M)
    a = [simplify(w[k]* sum([(2*n-1)*legendre(n-1,x[k])*f[j,n] for n=1:M]))  for j=1:J, k=1:M ]
    a,x
end

coeffs_a_c (generic function with 2 methods)

## $p=4$, $J=2$

In [5]:
p=4

4

In [6]:
J=2

2

In [7]:
C=[(-1)^(n-1)*(2*n-1)*sum([(-1)^k//(q+k+1)*binomial(n-1,k)*binomial(n+k-1,k) for k=0:n-1]) for q=0:p-1, n=1:p]

4×4 Array{Rational{Int64},2}:
 1//1  0//1   0//1  0//1 
 1//2  1//2   0//1  0//1 
 1//3  1//2   1//6  0//1 
 1//4  9//20  1//4  1//20

In [8]:
B = gen_hall_basis_elements_of_grades_leq(p)

7-element Array{Array{Int64,1},1}:
 [0]      
 [1]      
 [2]      
 [0, 1]   
 [3]      
 [0, 2]   
 [0, 0, 1]

In [9]:
eqs, vars=gen_CFET_order_conditions(B, J)
eqs=[expand(eq) for eq in eqs]

7-element Array{Giac.giac_SYMB,1}:
 b1_0+b2_0-1                                                            
 b1_1+b2_1-1/2                                                          
 b1_2+b2_2-1/3                                                          
 1/2*b1_0*b1_1+b1_1*b2_0+1/2*b2_0*b2_1-1/6                              
 b1_3+b2_3-1/4                                                          
 1/2*b1_0*b1_2+b1_2*b2_0+1/2*b2_0*b2_2-1/12                             
 1/2*b1_1*b2_0^2+1/6*b1_0^2*b1_1+1/6*b2_0^2*b2_1+1/2*b1_0*b1_1*b2_0-1/24

In [10]:
b=[giac(string("b",j,"_",q)) for j=1:J, q=0:p-1]

2×4 Array{Giac.giac_IDNT,2}:
 b1_0  b1_1  b1_2  b1_3
 b2_0  b2_1  b2_2  b2_3

In [11]:
f=[giac(string("f",j,"_",n)) for j=1:J, n=1:p]

2×4 Array{Giac.giac_IDNT,2}:
 f1_1  f1_2  f1_3  f1_4
 f2_1  f2_2  f2_3  f2_4

In [12]:
subs=reshape([sum([C[q+1,n]*f[j,n] for n=1:p]) for  q=0:p-1, j=1:J], p*J)

8-element Array{Giac.giac,1}:
 f1_1                                 
 1/2*f1_1+1/2*f1_2                    
 1/3*f1_1+1/2*f1_2+1/6*f1_3           
 1/4*f1_1+9/20*f1_2+1/4*f1_3+1/20*f1_4
 f2_1                                 
 1/2*f2_1+1/2*f2_2                    
 1/3*f2_1+1/2*f2_2+1/6*f2_3           
 1/4*f2_1+9/20*f2_2+1/4*f2_3+1/20*f2_4

In [13]:
vars

8-element Array{Giac.giac_IDNT,1}:
 b1_0
 b1_1
 b1_2
 b1_3
 b2_0
 b2_1
 b2_2
 b2_3

In [14]:
eqs1 = [expand(subst(eq, vars, subs )) for eq in eqs]

7-element Array{Giac.giac_SYMB,1}:
 f1_1+f2_1-1                                                                                                                      
 1/2*f1_1+1/2*f1_2+1/2*f2_1+1/2*f2_2-1/2                                                                                          
 1/3*f1_1+1/2*f1_2+1/6*f1_3+1/3*f2_1+1/2*f2_2+1/6*f2_3-1/3                                                                        
 1/4*f1_1^2+1/4*f2_1^2+1/4*f1_1*f1_2+1/2*f1_1*f2_1+1/2*f1_2*f2_1+1/4*f2_1*f2_2-1/6                                                
 1/4*f1_1+9/20*f1_2+1/4*f1_3+1/20*f1_4+1/4*f2_1+9/20*f2_2+1/4*f2_3+1/20*f2_4-1/4                                                  
 1/6*f1_1^2+1/6*f2_1^2+1/4*f1_1*f1_2+1/12*f1_1*f1_3+1/3*f1_1*f2_1+1/2*f1_2*f2_1+1/6*f1_3*f2_1+1/4*f2_1*f2_2+1/12*f2_1*f2_3-1/12   
 1/12*f1_1^3+1/12*f2_1^3+1/4*f1_1*f2_1^2+1/4*f1_2*f2_1^2+1/12*f1_1^2*f1_2+1/4*f1_1^2*f2_1+1/12*f2_1^2*f2_2+1/4*f1_1*f1_2*f2_1-1/24

In [15]:
ff = zeros(Rational{Int},J,p)
ff[1,1] = 1//2
ff[1,2] = -1//3
ff[2,1] = 1//2
ff[2,2] = 1//3
ff

2×4 Array{Rational{Int64},2}:
 1//2  -1//3  0//1  0//1
 1//2   1//3  0//1  0//1

In [16]:
[simplify(subst(eq, reshape(f,p*J),  reshape(ff,p*J))) for eq in eqs1]

7-element Array{Giac.giac_INT_,1}:
 0
 0
 0
 0
 0
 0
 0

In [17]:
a,c = coeffs_a_c(p,J,ff,2)

Warning, argument is not an equation, solving 6*x^2-6*x+1=0


(Giac.giac_SYMB[(2*sqrt(3)+3)/12 (-2*sqrt(3)+3)/12; (-2*sqrt(3)+3)/12 (2*sqrt(3)+3)/12], Giac.giac_SYMB[1/6*(-sqrt(3)+3), 1/6*(sqrt(3)+3)])

## $p=6$, $J=6$

In [18]:
p=6

6

In [19]:
J=6

6

In [20]:
C=[(-1)^(n-1)*(2*n-1)*sum([(-1)^k//(q+k+1)*binomial(n-1,k)*binomial(n+k-1,k) for k=0:n-1]) for q=0:p-1, n=1:p]

6×6 Array{Rational{Int64},2}:
 1//1  0//1    0//1   0//1   0//1   0//1  
 1//2  1//2    0//1   0//1   0//1   0//1  
 1//3  1//2    1//6   0//1   0//1   0//1  
 1//4  9//20   1//4   1//20  0//1   0//1  
 1//5  2//5    2//7   1//10  1//70  0//1  
 1//6  5//14  25//84  5//36  1//28  1//252

In [21]:
B = gen_hall_basis_elements_of_grades_leq(p)

22-element Array{Array{Int64,1},1}:
 [0]            
 [1]            
 [2]            
 [0, 1]         
 [3]            
 [0, 2]         
 [0, 0, 1]      
 [4]            
 [0, 3]         
 [1, 2]         
 [0, 0, 2]      
 [0, 1, 1]      
 [0, 0, 0, 1]   
 [5]            
 [0, 4]         
 [1, 3]         
 [0, 0, 3]      
 [0, 1, 2]      
 [0, 2, 1]      
 [0, 0, 0, 2]   
 [0, 0, 1, 1]   
 [0, 0, 0, 0, 1]

In [22]:
eqs, vars=gen_CFET_order_conditions(B, J)
eqs=[expand(eq) for eq in eqs]

22-element Array{Giac.giac_SYMB,1}:
 b1_0+b2_0+b3_0+b4_0+b5_0+b6_0-1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [23]:
b=[giac(string("b",j,"_",q)) for j=1:J, q=0:p-1]

6×6 Array{Giac.giac_IDNT,2}:
 b1_0  b1_1  b1_2  b1_3  b1_4  b1_5
 b2_0  b2_1  b2_2  b2_3  b2_4  b2_5
 b3_0  b3_1  b3_2  b3_3  b3_4  b3_5
 b4_0  b4_1  b4_2  b4_3  b4_4  b4_5
 b5_0  b5_1  b5_2  b5_3  b5_4  b5_5
 b6_0  b6_1  b6_2  b6_3  b6_4  b6_5

In [24]:
f=[giac(string("f",j,"_",n)) for j=1:J, n=1:p]

6×6 Array{Giac.giac_IDNT,2}:
 f1_1  f1_2  f1_3  f1_4  f1_5  f1_6
 f2_1  f2_2  f2_3  f2_4  f2_5  f2_6
 f3_1  f3_2  f3_3  f3_4  f3_5  f3_6
 f4_1  f4_2  f4_3  f4_4  f4_5  f4_6
 f5_1  f5_2  f5_3  f5_4  f5_5  f5_6
 f6_1  f6_2  f6_3  f6_4  f6_5  f6_6

In [25]:
subs=reshape([sum([C[q+1,n]*f[j,n] for n=1:p]) for  q=0:p-1, j=1:J], p*J)

36-element Array{Giac.giac,1}:
 f1_1                                                        
 1/2*f1_1+1/2*f1_2                                           
 1/3*f1_1+1/2*f1_2+1/6*f1_3                                  
 1/4*f1_1+9/20*f1_2+1/4*f1_3+1/20*f1_4                       
 1/5*f1_1+2/5*f1_2+2/7*f1_3+1/10*f1_4+1/70*f1_5              
 1/6*f1_1+5/14*f1_2+25/84*f1_3+5/36*f1_4+1/28*f1_5+1/252*f1_6
 f2_1                                                        
 1/2*f2_1+1/2*f2_2                                           
 1/3*f2_1+1/2*f2_2+1/6*f2_3                                  
 1/4*f2_1+9/20*f2_2+1/4*f2_3+1/20*f2_4                       
 1/5*f2_1+2/5*f2_2+2/7*f2_3+1/10*f2_4+1/70*f2_5              
 1/6*f2_1+5/14*f2_2+25/84*f2_3+5/36*f2_4+1/28*f2_5+1/252*f2_6
 f3_1                                                        
 ⋮                                                           
 f5_1                                                        
 1/2*f5_1+1/2*f5_2                     

In [26]:
eqs1 = [expand(subst(eq, vars, subs )) for eq in eqs]

22-element Array{Giac.giac_SYMB,1}:
 f1_1+f2_1+f3_1+f4_1+f5_1+f6_1-1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [27]:
ff = zeros(Float64,J,p)

6×6 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

In [28]:
ff[1,1] = 0.16 
ff[1,2] = 0.15101538937746543493 
ff[1,3] = 0.13304616813239630479
ff[2,1] = -0.22738164742696330169 
ff[2,2] = -0.087654259755115431662 
ff[2,3] = 0.069919836812656575583
ff[3,1] = 1/2 -ff[1,1] - ff[2,1] 
ff[3,2] = 0.21035154512209824847 
ff[3,3] = -ff[1,3] - ff[2,3]

ff[6,1] = ff[1,1]
ff[6,2] = -ff[1,2]
ff[6,3] = ff[1,3]
ff[5,1] = ff[2,1]
ff[5,2] = -ff[2,2]
ff[5,3] = ff[2,3]
ff[4,1] = ff[3,1]
ff[4,2] = -ff[3,2]
ff[4,3] = ff[3,3]
ff=flipdim(ff, 1)

6×6 Array{Float64,2}:
  0.16      -0.151015    0.133046   0.0  0.0  0.0
 -0.227382   0.0876543   0.0699198  0.0  0.0  0.0
  0.567382  -0.210352   -0.202966   0.0  0.0  0.0
  0.567382   0.210352   -0.202966   0.0  0.0  0.0
 -0.227382  -0.0876543   0.0699198  0.0  0.0  0.0
  0.16       0.151015    0.133046   0.0  0.0  0.0

In [29]:
[(subst(eq, reshape(f,p*J),  reshape(ff,p*J))) for eq in eqs1]

22-element Array{Giac.giac_DOUBLE_,1}:
 -1.06581410364e-14
 -3.5527136788e-15 
 -3.5527136788e-15 
 -1.42108547152e-14
 -2.6645352591e-15 
 -1.46549439251e-14
 -1.64313007645e-14
 -8.881784197e-16  
 -9.32587340685e-15
 -1.68753899743e-14
 -1.59872115546e-14
 -1.87627691162e-14
 -9.65894031424e-15
 -8.881784197e-16  
 -6.43929354283e-15
 -1.26565424807e-14
 -1.06026298852e-14
 -2.13162820728e-14
 -2.40918396344e-14
 -8.04911692853e-15
 -9.82547376793e-15
 -4.32986979604e-15

In [30]:
a,c = coeffs_a_c(p,J,ff,3)

Warning, argument is not an equation, solving 20*x^3-30*x^2+12*x-1=0


(Giac.giac_DOUBLE_[0.215838996976 -0.0767179645916 0.0208789676158; -0.0808977963209 -0.178747217537 0.032263366431; … ; 0.032263366431 -0.178747217537 -0.0808977963209; 0.0208789676158 -0.0767179645916 0.215838996976], Any[1/10*(-sqrt(15)+5), 1//2, 1/10*(sqrt(15)+5)])

In [31]:
map(to_julia, a)

6×3 Array{Float64,2}:
  0.215839   -0.076718   0.020879 
 -0.0808978  -0.178747   0.0322634
  0.180628    0.477687  -0.0909342
 -0.0909342   0.477687   0.180628 
  0.0322634  -0.178747  -0.0808978
  0.020879   -0.076718   0.215839 

In [32]:
c

3-element Array{Any,1}:
   1/10*(-sqrt(15)+5)
 1//2                
   1/10*(sqrt(15)+5) 